In [1]:
import snn_train as snn
import numpy as np
import random
import matplotlib.pyplot as plt

#snn.input_params['num_classes'] = 2
#snn.input_params['lif_simulation_time'] = 20

snn.set_max_current()
snn.set_min_current()

print('Max encoding current: {}'.format(snn.input_params['max_enc_input_current']))
print('Min encoding current: {}'.format(snn.input_params['min_enc_input_current']))
print('Max mlp current: {}'.format(snn.input_params['max_mlp_input_current']))
print('Min mlp current: {}'.format(snn.input_params['min_mlp_input_current']))

Max encoding current: -27.524995834325125
Min encoding current: 0.9998637940269709
Max mlp current: -27.524995834325125
Min mlp current: 0.9998637940269709


# Inspeção rápida do modelo para saber se ele está classificando de fato ou não

## Carregando dados do arquivo txt: São 8 caracteres num formato 12x10, onde cada coluna corresponde a um caractere (logo, 8 colunas) e cada coluna possui 120 linhas (imagem vetorizada: 12x10=120).

In [2]:
from numpy import genfromtxt
data = genfromtxt('char8_12x10.txt', delimiter=',')
data.shape

(120, 8)

## Transformação em um tensor de dimensão 8x12x10: A primeira dimensão são os caracteres, enquanto que as duas dimensões internas são a imagem em formato matricial.

In [3]:
char_num = data.shape[1]

char_list = []

for i in range(char_num):
    char_vec_img = data[:, i]
    char_mat_img = char_vec_img.reshape((12, 10))
    char_list.append(char_mat_img)

char_data_set = np.array(char_list)
char_data_set

array([[[-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
        [-1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1.],
        [-1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.],
        [-1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.],
        [-1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.],
        [-1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.],
        [-1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.],
        [-1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.],
        [-1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.],
        [-1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.],
        [-1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1.],
        [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.]],

       [[-1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1.],
        [-1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1.],
        [-1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1.],
        [-1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1.],
        [-1., -1., -1.,  1.,  1.,  1.,

## Normalização da Base para Max-Min (0,1):

In [4]:
norm_dataset = (char_data_set -np.min(char_data_set))/(np.max(char_data_set)-np.min(char_data_set))

## Geração de amostras ruidosas para completar o dataset de treino e teste:

In [5]:
# Geração de um conjunto de probabilidades para aplicar o ruido na imagem
noise_prob_levels = np.arange(0, 0.10, 0.0005)

# Construção de uma base de amostras ruidosas
label_samples = []
for label in range(snn.input_params['num_classes']):
    samples = []
    for sample in range(len(noise_prob_levels)):
        noisy_img = snn.img_noise(norm_dataset[label], noise_prob_levels[i])
        samples.append(noisy_img)
    label_samples.append(samples)

sample_set = np.array(label_samples)
print(sample_set.shape)


# Separação da base de amostras em base de treino e base de teste, na proporção 80-20
split_ratio = 0.8

train_lower_idx = 0
train_upper_idx = int(sample_set.shape[1]*split_ratio)
test_lower_idx = int(sample_set.shape[1]*split_ratio)
test_upper_idx = test_lower_idx + int(sample_set.shape[1]*(1-split_ratio)) + 1

train_set = sample_set[:, train_lower_idx:train_upper_idx]
test_set = sample_set[:, test_lower_idx:test_upper_idx]

print(train_lower_idx, train_upper_idx, test_lower_idx, test_upper_idx)
print(train_set.shape, test_set.shape)


(8, 200, 12, 10)
0 160 160 200
(8, 160, 12, 10) (8, 40, 12, 10)


## Geração dos impulsos desejados:

### Definição das Correntes Máxima e Mínima

In [14]:
import math

R = snn.input_params['lif_enc_resistance']
EL = snn.input_params['lif_enc_rest_potential']
V0 = snn.input_params['lif_enc_start_potential']
tao = snn.input_params['lif_enc_tao']
Vth = snn.input_params['lif_enc_threshold']
T = snn.input_params['lif_simulation_time']

def current_lif (R, EL, V0, tao, Vth, t):
    return ((EL-V0)-((EL-Vth)*math.exp(t/tao)))/(R*(math.exp(t/tao)-1))

I_max = math.ceil(current_lif(R, EL, V0, tao, Vth, 1))
I_min = math.ceil(current_lif(R, EL, V0, tao, Vth, T))
print('Corrente Máxima (1 impulso por ms): {}nA'.format(I_max))
print('Corrente Mínima (1 impulso por Tms, onde T é o tempo total): {}nA'.format(I_min))

Corrente Máxima (1 impulso por ms): 33nA
Corrente Mínima (1 impulso por Tms, onde T é o tempo total): 5nA


### Geração das Correntes por rótulo, respeitando o intervalo [Imin, Imax]

In [15]:
def label_currents (Nc, Imax, Imin):
    I_labels = np.zeros(Nc)
    for r in range(Nc):
        ir = Imin + r*((Imax-Imin)/(Nc-1))
        I_labels[r] = ir
    return I_labels

I_labels = label_currents(snn.input_params['num_classes'], I_max, I_min)
I_labels

array([ 5.,  9., 13., 17., 21., 25., 29., 33.])

### Geração dos Impulsos a partir das correntes

In [16]:
desired_times = []
desired_spikes = []
#desired_count = []

for il in I_labels:
    spb, spt, _ = snn.integrate_and_fire_spikes(R, tao, EL, V0, Vth, il, T)
    desired_spikes.append(spb)
    desired_times.append(spt)

desired_times = np.array(desired_times)
desired_spikes = np.array(desired_spikes)

labels = random.sample(range(0,snn.input_params['num_classes']), snn.input_params['num_classes'])
snn.input_params['desired_spikes'] = desired_spikes[np.sort(labels)]
snn.input_params['desired_times'] = desired_times[np.sort(labels)]
snn.input_params['desired_times']

array([array([18, 37, 56, 75, 94]),
       array([ 7, 15, 23, 31, 39, 47, 55, 63, 71, 79, 87, 95]),
       array([ 5, 11, 17, 23, 29, 35, 41, 47, 53, 59, 65, 71, 77, 83, 89, 95]),
       array([ 4,  9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, 69, 74, 79, 84,
       89, 94, 99]),
       array([ 3,  7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47, 51, 55, 59, 63, 67,
       71, 75, 79, 83, 87, 91, 95, 99]),
       array([ 3,  7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47, 51, 55, 59, 63, 67,
       71, 75, 79, 83, 87, 91, 95, 99]),
       array([ 2,  5,  8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50,
       53, 56, 59, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 95, 98]),
       array([ 2,  5,  8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50,
       53, 56, 59, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 95, 98])],
      dtype=object)

In [ ]:
labels = random.sample(range(0,snn.input_params['num_classes']), snn.input_params['num_classes'])
# Expected Values for Spikes for classes
desired_range = snn.spike_interval_generator(number_of_classes=snn.input_params['num_classes'], simulation_time=snn.input_params['lif_simulation_time'])
desired_isi = snn.spike_isi_generator(snn.input_params['num_classes'], snn.input_params['lif_simulation_time'])
dsb, dst = snn.spike_times_generator_new(desired_range, desired_isi)
snn.input_params['desired_spikes'] = dsb[np.sort(labels)]
snn.input_params['desired_times'] = dst[np.sort(labels)]
#snn.input_params['desired_count'] = desired_count
snn.input_params['desired_times']

## Treinamento da SNN:

In [ ]:
train_set[:,0].shape

In [ ]:
snn.input_params['num_samples'] = train_set.shape[1]
snn.input_params['num_test_samples'] = test_set.shape[1]
snn.input_params['num_epochs'] = 2

training_error = snn.fit(train_set)
plt.plot(training_error)
plt.title("SNN Training Error")
plt.xlabel('Epochs') 
plt.ylabel('MSE')

In [ ]:
acc = snn.eval_accuracy(test_set)
acc = acc * 100